**by: Valeria Viridiana Pineda Romero**

# Time Series Clustering using the Google Mobility Report of Mexico

## Data Preprocessing

We performed data cleaning tasks initially, such as variable selection, data engineering, and handling missing data.

### Variable Selection

Firstly, we eliminated the variables: `iso_3166_2_code`, `sub_region_2`, `metro_area`, `census_fips_code`, and `country_region`. The variable `iso_3166_2_code` was eliminated to avoid multicollinearity as it repeated information from variable `sub_region_1`. The variable `sub_region_2` was deleted as it did not add valuable information about the observations. Finally, the variables `metro_area`, `census_fips_code`, and `country_region` were composed entirely of missing values; thus, these were also removed.

In [ ]:
import pandas as pd
from matplotlib import pyplot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from math import sqrt
from sklearn.metrics import mean_squared_error


In [ ]:
series = pd.read_csv('../input/mobility-report-in-mexico-covid19/2020_MX_Region_Mobility_Report.csv', header=0, index_col=0)
del series['sub_region_2'] # Same value for all instances
del series['metro_area'] # all missing data
del series['census_fips_code'] # all missing data
del series['country_region'] # all missing data
del series['iso_3166_2_code']

In [ ]:
series.describe()

In [ ]:
series.describe(include=[object])

### Data Engineering

Next, we set the `date` variable as a DateTime data type, and we proceeded to place it as the index of the data frame.

In [ ]:
series['date'] = pd.to_datetime(series['date']).dt.strftime('%d/%m/%Y')

In [ ]:
series.set_index('date', inplace=True)

### Missing Data

Moreover, we analyzed the missing data, which appeared in `sub_region_1` and `transit_stations_percent_change_from_baseline` variables. Variable `sub_region_1` was made up of 3.03% of missing data. However, according to the information given, missing data in `sub_region_1` represent data at a national level. Thus, missing values were imputed with the `National Level` category. 

In [ ]:
pd.DataFrame(series.isnull().sum()/len(series)*100,columns=["NA"])

Percent of Total Missing Values

In [ ]:
len(series[series.isnull().any(axis=1)])/len(series)*100

Replace NA in 'sub_region_1' with 'National Level'

In [ ]:
series['sub_region_1'] = series['sub_region_1'].fillna('National Level')

On the other hand, `transit_stations_percent_change_from_baseline` variable presents 1.75% of missing data. Hence, we proceeded to treat them. As a time series analysis, we could not eliminate the instances that present missing data, so we decided to interpolate them.

In [ ]:
series['transit_stations_percent_change_from_baseline'] = series['transit_stations_percent_change_from_baseline'].interpolate()


Total Missing Values

In [ ]:
len(series[series.isnull().any(axis=1)])/len(series)

## Modeling

For this stage we grouped the states to identify which of them follow a similar behavior in terms of mobility in workplaces. In this project we applied the Hierarchical Clustering with several methods: the Single Method with the Pearson Correlation, the Single Method with the Spearman Correlation, the Single Method with the Dynamic Time Warpping, and the Ward method with Euclidean distance. 

In [ ]:
State = ['Aguascalientes',
 'Baja California',
 'Baja California Sur',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Coahuila',
 'Colima',
 'Durango',
 'Guanajuato',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'Mexico City',
 'Michoacán',
 'Morelos',
 'Nayarit',
 'Nuevo Leon',
 'Oaxaca',
 'Puebla',
 'Querétaro',
 'Quintana Roo',
 'San Luis Potosi',
 'Sinaloa',
 'Sonora',
 'State of Mexico',
 'Tabasco',
 'Tamaulipas',
 'Tlaxcala',
 'Veracruz',
 'Yucatan',
 'Zacatecas']

In [ ]:
timeSeries = pd.DataFrame()
for i in State:
    data = series[series['sub_region_1']==i]
    data['workplaces_percent_change_from_baseline'].plot(label=i)
    timeSeries = timeSeries.append(data['workplaces_percent_change_from_baseline'])
pyplot.xticks(rotation=45)
pyplot.legend(bbox_to_anchor = (1, 1.2))
pyplot.title('Workplaces')
pyplot.show()
timeSeries.index = State

In [ ]:
import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import dendrogram, linkage

def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)
    
    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

### Ward's method

In [ ]:
Z = hac.linkage(timeSeries, method='ward',optimal_ordering=True)

In [ ]:
# set cut-off to 80
max_d = 250  # max_d as in max_distance

fancy_dendrogram(
    Z,
    leaf_rotation=90.,
    leaf_font_size=10.,
    show_contracted=True,
    annotate_above=5,  # useful in small plots so annotations don't overlap
    max_d=max_d
)

plt.show()

### Pearson Correlation

In [ ]:
# Clustering with Linkage
Z_pearson_correlation = hac.linkage(timeSeries, method='single', metric='correlation', optimal_ordering=True)

In [ ]:
fancy_dendrogram(
    Z_pearson_correlation,
    leaf_rotation=90.,
    leaf_font_size=10.,
    show_contracted=True,
    annotate_above=5,  # useful in small plots so annotations don't overlap
    max_d=max_d
)

plt.show()

### Spearman Correlation

In [ ]:
# Here we use spearman correlation
def my_metric(x, y):
    r = stats.spearmanr(x, y)[0]
    return 1 - r # correlation to distance: range 0 to 2 # IF THE CLUSTERS ARE THE SAME THE DISTANCE WILL BE 0

# Do the clustering    
Z_spearman_correlation = hac.linkage(timeSeries,  method='single', metric=my_metric, optimal_ordering=True)

In [ ]:
fancy_dendrogram(
    Z_spearman_correlation,
    leaf_rotation=90.,
    leaf_font_size=10.,
    show_contracted=True,
    annotate_above=5,  # useful in small plots so annotations don't overlap
    max_d=max_d
)

plt.show()

The Ward method with Euclidean distance generates the most balanced clusters. This method, says that the distance between two clusters, A and B, is how much the sum of squares will increase when we merge them. Given its balanced results, we decided to group the States in such form.

## Deployment

### Generating clusters with Ward's method

The graphs below show the three clusters formed with the Ward method with Euclidean distance.

In [ ]:
from scipy.cluster.hierarchy import fcluster


def print_clusters(ts, Z, k, plot=False):
    plt.figure(figsize=(5, 5))
    # k Number of clusters I'd like to extract
    results = fcluster(Z, k, criterion='maxclust')
    print(results)

    # check the results
    s = pd.Series(results)
    clusters = s.unique()
    
    new_order = series.index.unique()
    ts = ts.T.reindex(new_order, axis=0)

    for c in clusters:
        cluster_indeces = s[s==c].index
        print("Cluster %d number of entries %d" % (c, len(cluster_indeces)))
        if plot:
            ts.iloc[:,cluster_indeces].plot()
        plt.xticks(rotation=45)
        plt.legend(bbox_to_anchor = (1, 1))
        plt.show()

In [ ]:
print_clusters(timeSeries, Z, 3, plot=True)